In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from yfinance import Ticker
import plotly.offline as pyo
import nbformat as nb


In [2]:
# %pip install yfinance==0.2.26
# !mamba install bs4==4.10.0 -y
# %pip install lxml
%pip install --upgrade yfinance
%pip install --upgrade nbformat

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Question 1: Use yfinance to Extract Stock Data

Reset the index, save, and display the first five rows of the `gme_data` dataframe using the `head` function.
Upload a screenshot of the results and code from the beginning of Question 1 to the results below.

In [3]:
tesla = Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
tesla_data.head()
# type(tesla_data)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


Question 2: Use Webscraping to Extract Tesla Revenue Data

Display the last five rows of the `tesla_revenue` dataframe using the `tail` function. Upload a screenshot of
the results.

In [4]:
html_data = requests.get(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm").text
soup = BeautifulSoup(html_data, 'lxml')
table = soup.find("table")
tesla_revenue = pd.read_html(str(table))[0]
tesla_revenue.columns = ['Date', 'Revenue']
tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace(
    '.', '').str.replace('$', '')
print(tesla_revenue.tail())
# print(type(tesla_revenue))


    Date Revenue
8   2013   2,013
9   2012     413
10  2011     204
11  2010     117
12  2009     112


Question 3: Use yfinance to Extract Stock Data

Reset the index, save, and display the first five rows of the `gme_data` dataframe using the `head` function.
Upload a screenshot of the results and code from the beginning of Question 1 to the results below.

In [5]:
gme = Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()
# type(tesla_data)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


Question 4: Use Webscraping to Extract GME Revenue Data

Display the last five rows of the `gme_revenue` dataframe using the `tail` function. Upload a screenshot of the
results.

In [6]:
html_data = requests.get(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html").text
soup = BeautifulSoup(html_data, 'lxml')
table = soup.find('table')
gme_revenue = pd.read_html(str(table))[0]
gme_revenue.columns = ['Date', 'Revenue']
gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace(
    '.', '').str.replace('$', '')
print(gme_revenue.tail())


    Date Revenue
11  2009   8,806
12  2008   7,094
13  2007   5,319
14  2006   3,092
15  2005   1,843


Question 5: Plot Tesla Stock Graph

Use the `make_graph` function to graph the Tesla Stock Data, also provide a title for the graph.
Upload a screenshot of your results.

In [7]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=(
        "Historical Share Price", "Historical Revenue"), vertical_spacing=.3)
    
    # Convert 'Date' columns to Timestamp objects if they are not already in that format
    stock_data['Date'] = pd.to_datetime(stock_data['Date'], infer_datetime_format=True)
    revenue_data['Date'] = pd.to_datetime(revenue_data['Date'], infer_datetime_format=True)
    
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    
    fig.add_trace(go.Scatter(x=stock_data_specific['Date'], y=stock_data_specific['Close'].astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=revenue_data_specific['Date'], y=revenue_data_specific['Revenue'].astype("float"), name="Revenue"), row=2, col=1)
    
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    
    fig.update_layout(showlegend=False, height=900, title=stock, xaxis_rangeslider_visible=True)
    pyo.iplot(fig)


In [8]:
tesla_revenue['Revenue'] = tesla_revenue['Revenue'].astype(str).str.replace(',', '')
make_graph(tesla_data, tesla_revenue, 'Tesla')

5.9.1


C:\Users\KIIT\AppData\Local\Temp\ipykernel_27564\704581503.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\KIIT\AppData\Local\Temp\ipykernel_27564\704581503.py:7: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



Question 6: Plot GameStop Stock Graph

Use the `make_graph` function to graph the GameStop Stock Data, also provide a title for the graph.
Upload a screenshot of your results.

In [9]:
gme_revenue['Revenue'] = gme_revenue['Revenue'].astype(str).str.replace(',', '')
make_graph(gme_data, gme_revenue, 'GameStop')

C:\Users\KIIT\AppData\Local\Temp\ipykernel_27564\704581503.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\KIIT\AppData\Local\Temp\ipykernel_27564\704581503.py:7: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

